In [5]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import missingno as mn
import seaborn as sb
from scipy.stats import chi2_contingency
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split



#Creacion del modelo con RLM
#modelo predictivo que ayuda a entender la demanda de bicicletas en distintas circunstancias 

#PRIMER MODELO
#variables a incluir que influyen de acuerdo al analisis exploratorio de los datos
#-Hora del dia, temperatura, visibilidad, radiacion solar, estacion, dia festivo.



#Importar datos desde el archivo de CSV
archivo = "C:/Users/natal/OneDrive/Documentos/U/2024-2/Analitica/PROYECTO1/datoslimpios.csv"
datos_completos = pd.read_csv(archivo)

#volver a definir dummies para las variables categóricas
seasons_dummies = pd.get_dummies(datos_completos['Seasons'], prefix='es_')
holiday_dummies = pd.get_dummies(datos_completos['Holiday'], prefix='es_')

# Concatenar las dummies con el DataFrame original
datos = pd.concat([datos_completos, seasons_dummies, holiday_dummies], axis=1)

# Eliminar las columnas originales categóricas ya que están representadas por las dummies
datos = datos.drop(['Seasons', 'Holiday'], axis=1)
print(datos.columns)

Index(['Date', 'Rented Bike Count', 'Hour', 'Temperature(C)', 'Humidity(%)',
       'Wind speed (m/s)', 'Visibility (10m)', 'Dew point temperature(C)',
       'Solar Radiation (MJ/m2)', 'Rainfall(mm)', 'Snowfall (cm)',
       'Functioning Day', 'es__Autumn', 'es__Spring', 'es__Summer',
       'es__Winter', 'es__Holiday', 'es__No Holiday'],
      dtype='object')


In [6]:
#modelo con division de datos pata entrenamiento 
## Valores x y y,
from sklearn.linear_model import LinearRegression

X = datos.loc[:, ['Hour', 'Temperature(C)','Visibility (10m)','Solar Radiation (MJ/m2)',
       'es__Autumn', 'es__Spring', 'es__Summer', 'es__Winter','es__Holiday','es__No Holiday']]

y = datos.loc[:, ['Rented Bike Count']]


# Creamos conjuntos de datos para entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=1)

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Entrenamos el modelo
lr.fit(X_train, y_train)

# Predecimos usando los datos de prueba
y_pred = lr.predict(X_test)

# Puntaje del modelo (coeficiente de determinación R^2)
puntaje = lr.score(X_test, y_test)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_[0]  # Convierte el intercepto a un valor escalar

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes:")
print(coef_df)
print(f"Intercepto: {intercepto}")
print('\n')
# Mostrar el puntaje del modelo
print(f"coeficiente de determinación R^2: {puntaje}")

Coeficientes:
                         Coeficiente
Hour                       31.818712
Temperature(C)             21.323944
Visibility (10m)            0.156908
Solar Radiation (MJ/m2)    21.642197
es__Autumn                 70.539833
es__Spring                 51.220220
es__Summer                 17.545234
es__Winter               -139.305286
es__Holiday               -71.515132
es__No Holiday             71.515132
Intercepto: -238.73871121353034


coeficiente de determinación R^2: 0.4453893424614761


In [7]:
from sklearn.model_selection import cross_val_score


#modelo con  validacion cruzada 

# Valores X y Y
X = datos.loc[:, ['Hour', 'Temperature(C)','Visibility (10m)','Solar Radiation (MJ/m2)',
       'es__Autumn', 'es__Spring', 'es__Summer', 'es__Winter','es__Holiday','es__No Holiday']]

y = datos.loc[:, 'Rented Bike Count']

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Realizamos validación cruzada con 5 particiones (5-fold cross-validation)
scores = cross_val_score(lr, X, y, cv=5, scoring='r2')  # cv=5 indica 5 particiones



# Entrenamos el modelo con todo el conjunto de datos para obtener los coeficientes finales
lr.fit(X, y)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_  # Ya es un escalar, no necesita indexado

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes del modelo final:")
print(coef_df)
print(f"Intercepto del modelo final: {intercepto}")

# Calcular el R² del modelo final (entrenado en todo el conjunto de datos)
r2_final = lr.score(X, y)
print(f"Coeficiente de determinación R² del modelo final con visibility: {r2_final}")


Coeficientes del modelo final:
                         Coeficiente
Hour                       31.790811
Temperature(C)             21.801415
Visibility (10m)            0.158414
Solar Radiation (MJ/m2)    21.438972
es__Autumn                 70.655931
es__Spring                 48.776857
es__Summer                 13.339429
es__Winter               -132.772216
es__Holiday               -65.016126
es__No Holiday             65.016126
Intercepto del modelo final: -240.81056362795186
Coeficiente de determinación R² del modelo final con visibility: 0.443899719354765


In [8]:
#modelo 2 validacion cruzada sin visibility
# Valores X y Y
X = datos.loc[:, ['Hour', 'Temperature(C)','Solar Radiation (MJ/m2)',
       'es__Autumn', 'es__Spring', 'es__Summer', 'es__Winter','es__Holiday','es__No Holiday']]

y = datos.loc[:, 'Rented Bike Count']

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Realizamos validación cruzada con 5 particiones (5-fold cross-validation)
scores = cross_val_score(lr, X, y, cv=5, scoring='r2')  

# Entrenamos el modelo con todo el conjunto de datos para obtener los coeficientes finales
lr.fit(X, y)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_  # Ya es un escalar, no necesita indexado

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes del modelo final:")
print(coef_df)
print(f"Intercepto del modelo final: {intercepto}")

# Calcular el R² del modelo final (entrenado en todo el conjunto de datos)
r2_final = lr.score(X, y)
print(f"Coeficiente de determinación R² del modelo final sin visibility: {r2_final}")


Coeficientes del modelo final:
                         Coeficiente
Hour                       33.086884
Temperature(C)             20.420178
Solar Radiation (MJ/m2)    41.995026
es__Autumn                 92.913540
es__Spring                 15.667053
es__Summer                 38.990150
es__Winter               -147.570743
es__Holiday               -58.246439
es__No Holiday             58.246439
Intercepto del modelo final: -15.92412942718363
Coeficiente de determinación R² del modelo final sin visibility: 0.42332571452944834


In [9]:
#modelo 3 validacion cruzada sin visibility y sin dummies holiday
# Valores X y Y
X = datos.loc[:, ['Hour', 'Temperature(C)','Solar Radiation (MJ/m2)',
       'es__Autumn', 'es__Spring', 'es__Summer', 'es__Winter']]

y = datos.loc[:, 'Rented Bike Count']

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Realizamos validación cruzada con 5 particiones (5-fold cross-validation)
scores = cross_val_score(lr, X, y, cv=5, scoring='r2')  # cv=5 indica 5 particiones



# Entrenamos el modelo con todo el conjunto de datos para obtener los coeficientes finales
lr.fit(X, y)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_  # Ya es un escalar, no necesita indexado

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes del modelo final:")
print(coef_df)
print(f"Intercepto del modelo final: {intercepto}")

# Calcular el R² del modelo final (entrenado en todo el conjunto de datos)
r2_final = lr.score(X, y)
print(f"Coeficiente de determinación R² del modelo final sin visibility y sin holiday: {r2_final}")

Coeficientes del modelo final:
                         Coeficiente
Hour                       33.157828
Temperature(C)             20.061829
Solar Radiation (MJ/m2)    42.313406
es__Autumn                 92.771507
es__Spring                 17.688971
es__Summer                 47.105863
es__Winter               -157.566341
Intercepto del modelo final: 40.14181706786553
Coeficiente de determinación R² del modelo final sin visibility y sin holiday: 0.421827339184351


In [10]:
#modelo 4 (final) con  validacion cruzada 

# Valores X y Y
X = datos.loc[:, ['Hour', 'Temperature(C)','Visibility (10m)','Solar Radiation (MJ/m2)',
       'es__Autumn', 'es__Spring', 'es__Summer', 'es__Winter']]

y = datos.loc[:, 'Rented Bike Count']

# Creamos el modelo de regresión lineal
lr = LinearRegression()

# Realizamos validación cruzada con 5 particiones (5-fold cross-validation)
scores = cross_val_score(lr, X, y, cv=5, scoring='r2')  # cv=5 indica 5 particiones


# Entrenamos el modelo con todo el conjunto de datos para obtener los coeficientes finales
lr.fit(X, y)

# Obtener los coeficientes (betas) y el intercepto
coeficientes = lr.coef_.flatten()  # Aplana el array 2D a 1D
intercepto = lr.intercept_  # Ya es un escalar, no necesita indexado

# Crear un DataFrame para mostrar los coeficientes con sus nombres de columna
coef_df = pd.DataFrame(coeficientes, index=X.columns, columns=['Coeficiente'])

print("Coeficientes del modelo final:")
print(coef_df)
print(f"Intercepto del modelo final: {intercepto}")

# Calcular el R² del modelo final (entrenado en todo el conjunto de datos)
r2_final = lr.score(X, y)
print(f"Coeficiente de determinación R² del modelo final con visibility sin holiday: {r2_final}")

Coeficientes del modelo final:
                         Coeficiente
Hour                       31.882156
Temperature(C)             21.388773
Visibility (10m)            0.156919
Solar Radiation (MJ/m2)    21.988036
es__Autumn                 70.707635
es__Spring                 50.719031
es__Summer                 22.631603
es__Winter               -144.058269
Intercepto del modelo final: -176.1671628983787
Coeficiente de determinación R² del modelo final con visibility sin holiday: 0.44203464136420223
